In [1]:
import pandas as pd
import numpy as np


df = pd.read_csv('/Users/dhruvpatel/Desktop/projects/DealPredection/data/vehicles.csv')

print(df.head())

           id                                                url  \
0  7222695916  https://prescott.craigslist.org/cto/d/prescott...   
1  7218891961  https://fayar.craigslist.org/ctd/d/bentonville...   
2  7221797935  https://keys.craigslist.org/cto/d/summerland-k...   
3  7222270760  https://worcester.craigslist.org/cto/d/west-br...   
4  7210384030  https://greensboro.craigslist.org/cto/d/trinit...   

                   region                         region_url  price  year  \
0                prescott    https://prescott.craigslist.org   6000   NaN   
1            fayetteville       https://fayar.craigslist.org  11900   NaN   
2            florida keys        https://keys.craigslist.org  21000   NaN   
3  worcester / central MA   https://worcester.craigslist.org   1500   NaN   
4              greensboro  https://greensboro.craigslist.org   4900   NaN   

  manufacturer model condition cylinders  ... size  type paint_color  \
0          NaN   NaN       NaN       NaN  ...  NaN   NaN

# EDA

In [2]:
df.columns

Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status',
       'transmission', 'VIN', 'drive', 'size', 'type', 'paint_color',
       'image_url', 'description', 'county', 'state', 'lat', 'long',
       'posting_date'],
      dtype='object')

In [3]:
## drop the columns that are not needed 
df_cleaned=df
df_cleaned=df_cleaned.drop(['url','image_url','county','VIN','size','condition'],axis=1) 

In [4]:
df_cleaned.isnull().sum()

id                   0
region               0
region_url           0
price                0
year              1205
manufacturer     17646
model             5277
cylinders       177678
fuel              3013
odometer          4400
title_status      8242
transmission      2556
drive           130567
type             92858
paint_color     130203
description         70
state                0
lat               6549
long              6549
posting_date        68
dtype: int64

In [5]:
# dropping rows due to NAs 

df_cleaned =df_cleaned.dropna(subset=['year','description','fuel','odometer','lat','long','transmission','model'])

In [6]:
df_cleaned.isnull().sum()

id                   0
region               0
region_url           0
price                0
year                 0
manufacturer     15836
model                0
cylinders       168709
fuel                 0
odometer             0
title_status      6778
transmission         0
drive           124295
type             88728
paint_color     123699
description          0
state                0
lat                  0
long                 0
posting_date         0
dtype: int64

In [7]:
len(df)-len(df_cleaned)

20827

# Filling missing values 

In [8]:
# filling missing values in title status with 'missing' 

df_cleaned['title_status'] = df_cleaned['title_status'].fillna('missing')
df_cleaned.isnull().sum()

id                   0
region               0
region_url           0
price                0
year                 0
manufacturer     15836
model                0
cylinders       168709
fuel                 0
odometer             0
title_status         0
transmission         0
drive           124295
type             88728
paint_color     123699
description          0
state                0
lat                  0
long                 0
posting_date         0
dtype: int64

In [9]:
df_cleaned['transmission'] = df_cleaned['transmission'].fillna('other')

In [10]:
# filling null values from the model. 
from model_extract import process_car_dataset

featured_eng=process_car_dataset(df_cleaned)


Starting data extraction and cleaning...
Data extraction and cleaning completed!


In [11]:
# increase = featured_eng['drive'].value_counts() - df_cleaned['drive'].value_counts()
# print(increase)

In [12]:
# null values reduced 
featured_eng.isnull().sum() - df_cleaned.isnull().sum()

id                  0
region              0
region_url          0
price               0
year                0
manufacturer   -12111
model               0
cylinders      -30367
fuel                0
odometer            0
title_status        0
transmission        0
drive          -71030
type           -40512
paint_color         0
description         0
state               0
lat                 0
long                0
posting_date        0
dtype: int64

In [13]:
featured_eng.isnull().sum()

id                   0
region               0
region_url           0
price                0
year                 0
manufacturer      3725
model                0
cylinders       138342
fuel                 0
odometer             0
title_status         0
transmission         0
drive            53265
type             48216
paint_color     123699
description          0
state                0
lat                  0
long                 0
posting_date         0
dtype: int64

In [14]:
# handling the maufacturer column  

# prinitng the values where manufecturer is null 

manufecturer_null = featured_eng[featured_eng['manufacturer'].isna()]

# printing the values where manufecturer is null 


manufecturer_null['model'].value_counts()





model
Janesville                          49
PROGRAMS as little as $500 DOWN!    48
ALL                                 35
chryler town and country            28
ALL MAKES                           28
                                    ..
Chevelle  s10                        1
HHR 1LT                              1
1992 corvette  convertible           1
94 mustang convertible gt 5.0        1
Paige Glenbrook Touring              1
Name: count, Length: 2366, dtype: int64

In [15]:
# dropping the rows where manufacturer is null 

featured_eng = featured_eng[featured_eng['manufacturer'].notna()]



# drive column cleaning 

In [16]:
featured_eng['drive'].value_counts()

drive
4wd                  182408
fwd                  104328
rwd                   59363
allwheeldrive          2883
frontwheeldrive        1285
rearwheeldrive          637
front wheel drive         1
Name: count, dtype: int64

In [17]:
from drive_cleaning import process_drive_column
featured_eng=process_drive_column(featured_eng, 'drive')


Starting drive column cleaning...
Drive column cleaning completed!
Updated distribution:
drive
4wd    185291
fwd    105614
rwd     60000
Name: count, dtype: int64
DRIVE COLUMN CLEANING REPORT

BEFORE CLEANING:
------------------------------
drive
4wd                  182408
fwd                  104328
rwd                   59363
allwheeldrive          2883
frontwheeldrive        1285
rearwheeldrive          637
front wheel drive         1
Name: count, dtype: int64

AFTER CLEANING:
------------------------------
drive
4wd    185291
fwd    105614
rwd     60000
Name: count, dtype: int64

CHANGES:
------------------------------
allwheeldrive: 2883 → 0 (-2883)
4wd: 182408 → 185291 (+2883)
rwd: 59363 → 60000 (+637)
front wheel drive: 1 → 0 (-1)
frontwheeldrive: 1285 → 0 (-1285)
rearwheeldrive: 637 → 0 (-637)
fwd: 104328 → 105614 (+1286)

VALIDATION:
------------------------------
Total rows: 402328
Null values: 51423
Unexpected values: []
Column is clean: True

Drive column cleaning complete

In [18]:
# filling null values from research :
from drive_cleaning import fill_missing_drive_from_reference


featured_eng = fill_missing_drive_from_reference(featured_eng,reference_file='/Users/dhruvpatel/Desktop/projects/DealPredection/data/models_with_drive.csv')






 

Loading reference data from: /Users/dhruvpatel/Desktop/projects/DealPredection/data/models_with_drive.csv
Loaded 83 model-drive mappings from reference file
Found 51423 rows with missing drive values
✓ Filled drive for model 'focus' → 'fwd'
✓ Filled drive for model 'optima' → 'fwd'
✓ Filled drive for model 'versa' → 'fwd'
✓ Filled drive for model 'monte carlo' → 'fwd'
✓ Filled drive for model 'accord ex-l' → 'fwd'
✓ Filled drive for model 'jetta' → 'fwd'
✓ Filled drive for model '200' → 'fwd'
✓ Filled drive for model 'mustang' → 'rwd'
✓ Filled drive for model 'impala lt' → 'fwd'
✓ Filled drive for model 'avalon' → 'fwd'
✓ Filled drive for model 'cooper s' → 'fwd'
✓ Filled drive for model 'accord' → 'fwd'
✓ Filled drive for model 'fusion se' → 'fwd'
✓ Filled drive for model 'lesabre' → 'fwd'
✓ Filled drive for model 'maxima' → 'fwd'
✓ Filled drive for model 'maxima' → 'fwd'
✓ Filled drive for model '200' → 'fwd'
✓ Filled drive for model '200' → 'fwd'
✓ Filled drive for model 'sonata' → 

In [19]:
featured_eng['drive'].isna().sum()

np.int64(38617)

In [20]:
featured_eng['drive'].value_counts()

drive
4wd    185291
fwd    116757
rwd     61663
Name: count, dtype: int64

In [21]:
featured_eng.isnull().sum()

id                   0
region               0
region_url           0
price                0
year                 0
manufacturer         0
model                0
cylinders       136673
fuel                 0
odometer             0
title_status         0
transmission         0
drive            38617
type             46908
paint_color     122057
description          0
state                0
lat                  0
long                 0
posting_date         0
dtype: int64

# Cleaning Model 

Clean in stages:
* Stage 1: Remove obvious junk (numbers, too short, too long)
* Stage 2: Extract core model from complex strings
* Stage 3: Standardize spelling and format
* Stage 4: Apply manufacturer-specific rules

In [22]:
# Stage 1 : Remove obvious junk (numbers, too short, too long)

from model_cleaning import remove_numerical_models

featured_eng=remove_numerical_models(featured_eng)

featured_eng['model'].value_counts().to_csv('model_counts.csv')






Debug: DataFrame shape before cleaning: (402328, 20)
Debug: Numerical mask sum: 11783
Debug: Length mask sum: 1651
Debug: Combined mask sum: 13434
Debug: Sample of rows to remove: [False, False, False, False, False, False, False, False, False, False]
Debug: DataFrame shape after cleaning: (388894, 20)
Step: remove_numerical_models
Rows removed due to numerical only: 11783
Rows removed due to length > 40: 1651
Total rows removed: 13434


In [ ]:
# stage 2 : 

In [ ]:
#creating a list of all the models for each manufecturer 

models_available=featured_eng.groupby(['manufacturer','model']).size().reset_index(name='count') 



,manufacturer,model,count
23219,toyota,- LUXURY SUV - ALL WHEEL DRIVE,2
23220,toyota,"155,037",1
23221,toyota,1966 ElCamino,1
23222,toyota,1985 FJ60,1
23223,toyota,1997 toy t100,1
...,...,...,...
24852,toyota,yaris se,3
24853,toyota,yaris se hatchback,1
24854,toyota,yaris se hatchback sedan,3
24855,toyota,yaris sedan,15


In [26]:
featured_eng['manufacturer'].unique()

array(['gmc', 'chevrolet', 'toyota', 'ford', 'jeep', 'nissan', 'ram',
       'mazda', 'cadillac', 'honda', 'dodge', 'lexus', 'jaguar', 'buick',
       'chrysler', 'volvo', 'audi', 'infiniti', 'lincoln', 'alfa-romeo',
       'subaru', 'scion', 'acura', 'hyundai', 'mercedes-benz', 'bmw',
       'mitsubishi', 'smart', 'volkswagen', 'porsche', 'kia', 'sterling',
       'freightliner', 'maserati', 'international', 'saab', 'suzuki',
       'ferrari', 'mini', 'pontiac', 'plymouth', 'oldsmobile', 'fiat',
       'rover', 'genesis', 'hummer', 'isuzu', 'hino', 'kenworth', 'tesla',
       'saturn', 'am general', 'mercury', 'willys', 'bentley', 'nash',
       'mg', 'triumph', 'peterbilt', 'geo', 'harley-davidson', 'amc',
       'kaiser', 'lamborghini', 'edsel', 'austin', 'rolls-royce',
       'datsun', 'studebaker', 'mclaren', 'polaris', 'mack', 'hudson',
       'desoto', 'lotus', 'aston-martin', 'crosley', 'packard',
       'western star', 'land rover', 'morris', 'eagle', 'daewoo',
       'daihats

In [98]:
models_available[ (models_available['manufacturer']=='daihatsu' ) | (models_available['manufacturer']=='vpg')  ].to_csv('isuzu_hino_kenworth.csv')

In [ ]:
df_cleaned['cylinders'].unique()

array(['8 cylinders', '6 cylinders', nan, '4 cylinders', '5 cylinders',
       'other', '3 cylinders', '10 cylinders', '12 cylinders'],
      dtype=object)

In [ ]:
daewoo_models = [
    "lanos"
]

daihatsu_models = [
    "atrai-turbo",
    "hijet",
    "rocky-se"
]

vpg_models = [
    "mv-1"
]

eagle_models = [
    "custom",
    "limited-4x4",
    "summit",
    "talon",
    "vision"
]

morris_models = [
    "minor",
    "minor-1000",
    "minor-woody"
]

land_rover_models = [
    "defender-110",
    "discovery",
    "discovery-i",
    "discovery-ii",
    "freelander",
    "lr2",
    "lr3",
    "lr4-hse",
    "range-rover",
    "range-rover-evoque",
    "range-rover-sport",
    "range-rover-velar"
]

packard_models = [
    "110",
    "120",
    "160",
    "caribbean",
    "clipper",
    "club-opera",
    "deluxe-8",
    "limo",
    "one-twenty",
    "patrician-limousine",
    "sedan",
    "super-clipper",
    "touring-sedan"
]

western_star_models = [
    "4700sd",
    "4900",
    "4900sa"
]
aston_martin_models = [
    "db7",
    "db7-volante",
    "db9",
    "db11",
    "rapide",
    "rapide-s",
    "v8-vantage",
    "v8-vantage-base",
    "v8-vantage-s",
    "vanquish",
    "vanquish-v12",
    "vantage"
]

crosley_models = [
    "wagon"
]

lotus_models = [
    "1960-elite",
    "1991-elan-se",
    "diva-roadster",
    "elan-s130",
    "elan-se-m100",
    "elise",
    "elite",
    "evora",
    "mazdaspeed-miata",
    "stalker-xl"
]
desoto_models = [
    "custom",
    "firedome"
]

hudson_models = [
    "commodore",
    "convertible",
    "hornet",
    "pickup",
    "super-six",
    "supersix-hornet",
    "terraplane"
]

mack_models = [
    "ch600",
    "ch613",
    "chn613",
    "cl700",
    "cl713",
    "ctp713",
    "cv713",
    "cv713-granite",
    "cv713-granite-dump-truck",
    "cv713-granite-tri-axle",
    "cxn612",
    "cxu612",
    "dm",
    "dm600",
    "granite",
    "granite-gu712",
    "granite-gu713",
    "gr64b",
    "gu713",
    "gu813",
    "leu-613-rear-loader",
    "md642",
    "midliner-ms250",
    "mr688p",
    "pinnacle-cxu600",
    "pinnacle-cxu613",
    "r-model",
    "r688st",
    "rb",
    "rd",
    "rd688s",
    "u-model",
    "vision-cxn613"
]

mclaren_models = [
    "570gt",
    "570gt-w-upgrades",
    "570s",
    "720s-performance",
    "asc-mclaren"
]

polaris_models = [
    "damon-workhorse",
    "ranger",
    "razor",
    "rzr",
    "slingshot",
    "slingshot-r",
    "slingshot-sl",
    "xp-4-turbo-rzr"
]

studebaker_models = [
    "avanti",
    "avanti-r1",
    "champion",
    "commander",
    "golden-hawk",
    "gt-hawk",
    "gt-hawk-torisimo",
    "hawk",
    "lark",
    "lark-cruiser",
    "pickup",
    "president",
    "r17",
    "speedster",
    "street-rod",
    "t-bucket",
    "truck"
]

aston_martin_models = [
    "db7",
    "db9", 
    "db11",
    "db7-volante",
    "rapide",
    "rapide-s",
    "v8-vantage",
    "v8-vantage-base",
    "v8-vantage-s",
    "vanquish",
    "vanquish-v12",
    "vantage"
]

datsun_models = [
    "120y",
    "1600-roadster",
    "2000-roadster", 
    "240z",
    "260z",
    "280-zx",
    "280z",
    "280zx",
    "280zx-turbo",
    "300zx",
    "nissan-datsun-720",
    "roadster",
    "roadster-2000",
    "truck"
]

lotus_models = [
    "1960-elite",
    "1991-elan-se",
    "diva-roadster",
    "elan-s130",
    "elan-se-m100",
    "elise",
    "elite",
    "evora",
    "mazdaspeed-miata",
    "stalker-xl"
]

austin_models = [
    "american-bantam",
    "cooper",
    "countryman",
    "devon",
    "healey",
    "healey-100-4",
    "healey-3000",
    "healey-bugeye-sprite",
    "healey-sprite",
    "nippy"
]

edsel_models = [
    "corsair",
    "pacer",
    "ranger"
]

rolls_royce_models = [
    "corniche",
    "dawn",
    "ghost",
    "phantom",
    "silver-seraph",
    "silver-shadow",
    "silver-spirit",
    "silver-spur",
    "wraith"
]


mg_models = [
    "bgt",
    "mga",
    "mgb",
    "midget",
    "tc",
    "td"
]
amc_models = [
    "ambassador",
    "amx",
    "cj5",
    "concord",
    "eagle",
    "hornet",
    "javelin",
    "m38",
    "marlin",
    "matador",
    "pacer",
    "rambler",
    "rebel",
    "spirit"
]

kaiser_models = [
    "deluxe",
    "henry-j",
    "jeepster",
    "m35",
    "manhattan",
    "willys"
]

kaiser_models = [
    "deluxe",
    "henry-j",
    "jeepster",
    "m35",
    "manhattan",
    "willys"
]


geo_models = [
    "metro",
    "prizm",
    "tracker"
]

harley_models = [
    "breakout",
    "dyna",
    "electra-glide",
    "fat-boy",
    "heritage",
    "road-glide",
    "road-king",
    "softail",
    "sportster",
    "street-glide",
    "tri-glide",
    "v-rod"
]

peterbilt_models = [
    "330",
    "335",
    "337",
    "348",
    "357",
    "367",
    "376",
    "377",
    "378",
    "379",
    "385",
    "386",
    "387",
    "388",
    "389",
    "567",
    "579",
    "cabover"
]



triumph_models = [
    "gt6",
    "spitfire",
    "stag",
    "tr250",
    "tr3",
    "tr4",
    "tr6",
    "tr7",
    "tr8"
]

nash_models = [
    "ambassador",
    "healey",
    "lafayette",
    "metropolitan",
    "rambler",
    "victoria"
]

am_general_models = [
    "dj5",
    "hummer",
    "humvee",
    "m1098",
    "m1233",
    "m151a2",
    "m49a2",
    "m923a1",
    "m998",
    "mv-1"
]
saturn_models = [
    "astra",
    "aura",
    "ion",
    "l-series",
    "outlook",
    "relay",
    "s-series",
    "sky",
    "vue"
]
tesla_models = [
    "model-3",
    "model-s",
    "model-x",
    "model-y"
]

bentley_models = [
    "arnage",
    "azure",
    "bentayga",
    "continental",
    "continental-gt",
    "continental-gtc",
    "continental-supersports",
    "eight",
    "flying-spur",
    "mark-vi",
    "mulsanne",
    "turbo-r"
]

mercury_models = [
    "bobcat",
    "capri",
    "comet",
    "cougar",
    "cyclone",
    "eight",
    "grand-marquis",
    "marauder",
    "mariner",
    "marquis",
    "medalist",
    "milan",
    "montclair",
    "montego",
    "monterey",
    "mountaineer",
    "mystique",
    "sable",
    "sun-valley",
    "villager",
    "zephyr"
]

willys_models = [
    "aero",
    "cj",
    "coupe",
    "jeepster",
    "m38",
    "mb",
    "overland",
    "pickup",
    "truck",
    "wagon",
    "whippet",
    "woodill-wildfire"
]


kenworth_models = [
    "k270",
    "t20",
    "t270",
    "t3",
    "t300", 
    "t370",
    "t440",
    "t600",
    "t660",
    "t680",
    "t700",
    "t800",
    "t880",
    "w900"
]
hino_models = [
    "155",
    "165", 
    "185",
    "195",
    "258",
    "268",
    "338"
]
isuzu_models = [
    "117",
    "amigo",
    "ascender", 
    "axiom",
    "frr",
    "ftr",
    "fvr",
    "hombre",
    "htr",
    "i-mark",
    "i290",
    "i350",
    "npr",
    "nqr",
    "nrp",
    "nrr",
    "pup",
    "rodeo",
    "trooper",
    "vehicross",
    "w5500"
]


rover_models = [
    "classic",
    "defender",
    "discovery",
    "discovery-sport",
    "evoque",
    "freelander", 
    "lr2",
    "lr3",
    "lr4",
    "p5",
    "range-rover",
    "range-rover-sport",
    "range-rover-velar",
    "santana",
    "series",
    "tc-2000"
]

hummer_models = [
    "h1",
    "h2",
    "h3",
    "h3t",
    "m998"
]

genesis_models = [
    "coupe",
    "g70",
    "g80",
    "g90",
    "genesis"
]

fiat_models = [
    "124-spider",
    "500",
    "500c",
    "500e",
    "500l",
    "500x",
    "850-spider",
    "campagnola",
    "pininfarina-spider",
    "spider",
    "spider-2000",
    "x1-9"
]
oldsmobile_models = [
    "442",
    "88",
    "98",
    "achieva",
    "alero",
    "aurora",
    "bravada",
    "cutlass",
    "cutlass-calais",
    "cutlass-ciera",
    "cutlass-salon",
    "cutlass-supreme",
    "delta-88",
    "dynamic-88",
    "eighty-eight",
    "f85",
    "hurst",
    "intrigue",
    "lss",
    "omega",
    "regency",
    "rocket-88",
    "series-76",
    "silhouette",
    "starfire",
    "super-88",
    "toronado",
    "torpedo-back",
    "type-r"
]
plymouth_models = [
    "barracuda",
    "belvedere",
    "business-coupe",
    "cambridge",
    "caravelle",
    "cranbrook",
    "cuda",
    "deluxe",
    "duster",
    "fury",
    "gran-fury",
    "grand-voyager",
    "gtx",
    "laser",
    "neon",
    "p4",
    "pa",
    "plaza",
    "prowler",
    "pt-105",
    "reliant",
    "road-runner",
    "satellite",
    "savoy",
    "scamp",
    "special-deluxe",
    "sport-fury",
    "suburban",
    "valiant",
    "volare",
    "voyager",
    "woodie-wagon"
]

models_pontiac = [
    "astre",
    "aztek",
    "bonneville",
    "catalina",
    "chieftain",
    "deluxe",
    "fiero",
    "firebird",
    "firehawk",
    "formula",
    "g3",
    "g5",
    "g6",
    "g8",
    "grand-am",
    "grand-prix",
    "grandville",
    "gto",
    "lemans",
    "montana",
    "parisienne",
    "redbird",
    "silver-streak",
    "solstice",
    "star-chief",
    "streamliner",
    "sunbird",
    "sunfire",
    "super-chief",
    "tempest",
    "torpedo",
    "torrent",
    "trans-am",
    "trans-sport",
    "ventura",
    "vibe"
]
mini_models = [
    "clubman",
    "convertible",
    "cooper",
    "countryman",
    "coupe",
    "hardtop",
    "john-cooper-works",
    "mark",
    "moke",
    "nova",
    "paceman",
    "roadster"
]
ferrari_models = [
    "348",
    "360",
    "456",
    "458",
    "488",
    "550",
    "575",
    "california",
    "f12",
    "f355",
    "f430",
    "f50",
    "f8",
    "ff",
    "mondial"
]
saab_models = [
    "9-2x",
    "9-3",
    "9-4x", 
    "9-5",
    "9-7x",
    "900",
    "9000",
    "93x",
    "sonett"
]

suzuki_models = [
    "aerio",
    "c90t",
    "cappuccino",
    "carry",
    "equator",
    "esteem",
    "forenza",
    "grand-vitara",
    "gsx-r600",
    "gsx-r750",
    "gsx-s750",
    "gsx1300r",
    "intruder",
    "kizashi",
    "reno",
    "samurai",
    "sfv650",
    "sidekick",
    "sx4",
    "verona",
    "vitara",
    "vl1500",
    "vl800",
    "wagon-r",
    "xl7"
]

maserati_models = [
    "ghibli",
    "granturismo", 
    "levante",
    "m128gt",
    "quattroporte",
    "spyder"
]
freightliner_models = [
    "125",
    "16m",
    "argosy",
    "c2",
    "cascadia",
    "century",
    "classic",
    "columbia",
    "coronado",
    "f650",
    "fc70",
    "fl106",
    "fl112",
    "fl50",
    "fl60",
    "fl70",
    "fl80",
    "flc120",
    "fld",
    "fld112",
    "fs65",
    "m2",
    "m55",
    "mt45",
    "mt55",
    "p700",
    "sportchassis",
    "sprinter",
    "unimog",
    "usf-1e",
    "xc-m"
]
models_sterling = [
    "5500",
    "9500", 
    "acterra",
    "bullet",
    "l-series",
    "l7500",
    "l8500",
    "lt7500",
    "lt8500",
    "sc8000"
]
kia_models = [
    "amanti",
    "borrego",
    "cadenza",
    "forte",
    "k5",
    "k900",
    "niro",
    "optima",
    "rio",
    "rondo",
    "sedona",
    "seltos",
    "sephia",
    "sol",
    "sorento",
    "soul",
    "spectra",
    "sportage",
    "stinger",
    "telluride"
]
porsche_models = [
    "356",
    "718-boxster",
    "718-cayman", 
    "911",
    "912",
    "914",
    "924",
    "928",
    "944",
    "996",
    "boxster",
    "cayenne",
    "cayman",
    "karmann-ghia",
    "macan",
    "panamera",
    "taycan"
]
volkswagen_models = [
    "arteon",
    "atlas",
    "beetle",
    "cabrio",
    "cc",
    "corrado",
    "e-golf",
    "eos",
    "eurovan",
    "fastback",
    "golf",
    "id4",
    "jetta",
    "jetta-sportwagen",
    "karmann-ghia",
    "passat",
    "phaeton",
    "r32",
    "rabbit",
    "routan",
    "squareback",
    "thing",
    "tiguan",
    "touareg",
    "vanagon",
    "baja"
]
smart_models = [
    "fortwo"
]
mitsubishi_models = [
    "3000gt",
    "delica",
    "diamante",
    "eclipse",
    "eclipse-cross",
    "endeavor",
    "fuso",
    "galant",
    "i-miev",
    "lancer",
    "lancer-evolution",
    "mighty-max",
    "minicab",
    "mirage",
    "montero",
    "montero-sport",
    "outlander",
    "outlander-sport",
    "pajero",
    "raider"
]
bmw_models = [
    "1-series",
    "2-series",
    "3-series",
    "4-series",
    "5-series",
    "6-series",
    "7-series",
    "8-series",
    "activehybrid-5",
    "alpina",
    "e30",
    "e36",
    "e39",
    "e46",
    "e60",
    "e90",
    "i3",
    "i8",
    "isetta",
    "m2",
    "m235i",
    "m240i",
    "m3",
    "m4",
    "m5",
    "m550i",
    "m6",
    "m8",
    "m850i",
    "x1",
    "x2",
    "x3",
    "x4",
    "x5",
    "x6",
    "x7",
    "z3",
    "z4"
]
mercedes_benz_models = [
    "a-class",
    "b-class",
    "c-class",
    "cl-class",
    "cla-class",
    "clk-class",
    "cls-class",
    "e-class",
    "g-class",
    "gl-class",
    "gla-class",
    "glb-class",
    "glc-class",
    "gle-class",
    "glk-class",
    "gls-class",
    "m-class",
    "ml-class",
    "r-class",
    "s-class",
    "sl-class",
    "slc-class",
    "slk-class",
    "amg-gt",
    "metris",
    "sprinter",
    "unimog",
    "190",
    "220",
    "240",
    "280",
    "300",
    "380",
    "420",
    "450",
    "500",
    "560"
]
hyundai_models = [
    "accent",
    "azera",
    "elantra",
    "entourage",
    "equus",
    "excel",
    "genesis",
    "ioniq",
    "kona",
    "nexo",
    "palisade",
    "santa-fe",
    "sonata",
    "tiburon",
    "tucson",
    "veloster",
    "venue",
    "veracruz",
    "xg350"
]
acura_models = [
    "cl",
    "ilx",
    "integra",
    "legend",
    "mdx",
    "nsx",
    "rdx",
    "rl",
    "rlx",
    "rsx",
    "slx",
    "tl",
    "tlx",
    "tsx",
    "zdx"
]
scion_models = [
    "fr-s",
    "ia",
    "im",
    "iq",
    "tc",
    "xa",
    "xb",
    "xd"
],

subaru_models = [
    "ascent",
    "baja",
    "brat",
    "brz",
    "crosstrek",
    "forester",
    "gl",
    "impreza",
    "justy",
    "legacy",
    "loyale",
    "outback",
    "svx",
    "tribeca",
    "wrx",
    "wrx-sti"
]
alfa_romeo_models = [
    "164",
    "e320",
    "giulia",
    "giulia-q4-awd",
    "giulia-quadrifoglio",
    "giulia-sport",
    "giulia-ti",
    "giulia-ti-q4-awd",
    "giulia-ti-sport",
    "gtv",
    "quad",
    "romeo",
    "romeo-spider",
    "spider",
    "spider-graduate",
    "spider-veloce",
    "stelvio",
    "stelvio-base",
    "stelvio-quadrifoglio",
    "stelvio-sport",
    "stelvio-sport-awd",
    "stelvio-sport-utility",
    "stelvio-ti",
    "stelvio-ti-lusso",
    "stelvio-ti-sport",
    "xcs-chassis"
]

lincoln_models = [
    "aviator",
    "blackwood",
    "capri",
    "continental",
    "corsair",
    "ls",
    "mark-iii",
    "mark-iv",
    "mark-lt",
    "mark-v",
    "mark-vi",
    "mark-vii",
    "mark-viii",
    "mkc",
    "mks",
    "mkt",
    "mkx",
    "mkz",
    "model-l",
    "nautilus",
    "navigator",
    "town-car",
    "town-coupe",
    "zephyr"
]
infiniti_models = [
    "ex35",
    "ex37",
    "fx35",
    "fx37",
    "fx45",
    "fx50",
    "g20",
    "g25",
    "g35",
    "g37",
    "i30",
    "i35",
    "j30",
    "jx35",
    "m30",
    "m35",
    "m37",
    "m45",
    "m56",
    "q40",
    "q45",
    "q50",
    "q60",
    "q70",
    "qx30",
    "qx4",
    "qx50",
    "qx56",
    "qx60",
    "qx70",
    "qx80"
]
audi_models = [
    "100",
    "5000",
    "80",
    "90",
    "a3",
    "a4",
    "a5",
    "a6",
    "a7",
    "a8",
    "allroad",
    "e-tron",
    "q3",
    "q5",
    "q7",
    "q8",
    "r8",
    "rs3",
    "rs4",
    "rs5",
    "rs6",
    "rs7",
    "s3",
    "s4",
    "s5",
    "s6",
    "s7",
    "s8",
    "sq5",
    "tt",
    "tts"
]
volvo_models = [
    "122s",
    "240",
    "245",
    "740",
    "850",
    "940",
    "960",
    "amazon",
    "c30",
    "c70",
    "commercial",
    "p1800",
    "pv544",
    "s40",
    "s60",
    "s70",
    "s80",
    "s90",
    "v40",
    "v50",
    "v60",
    "v70",
    "v90",
    "xc40",
    "xc60",
    "xc70",
    "xc90"
]
toyota_models = [
    "4Runner",
    "86",
    "Avalon",
    "C-HR",
    "Camry",
    "Celica",
    "Celsior",
    "Century",
    "Chinook",
    "Coaster",
    "Corolla",
    "Corona",
    "Cressida",
    "Cresta",
    "Crown",
    "Echo",
    "Elegante",
    "Estima",
    "FJ Cruiser",
    "GR Supra",
    "HiAce",
    "Highlander",
    "Hilux",
    "Land Cruiser",
    "Liteace",
    "MR2",
    "Matrix",
    "Mirai",
    "Paseo",
    "Pickup",
    "Previa",
    "Prius",
    "RAV4",
    "Scion",
    "Sequoia",
    "Sera",
    "Sienna",
    "Soarer",
    "Solara",
    "Stout",
    "Supra",
    "T100",
    "Tacoma",
    "Tercel",
    "Townace",
    "Trekker",
    "Tundra",
    "Van",
    "Venza",
    "Yaris"
]


ford_models = [
    "Aerostar",
    "Aspire",
    "Bronco",
    "C-Max",
    "Contour",
    "Convertible",
    "Coupe",
    "Courier",
    "Crown Victoria",
    "Custom",
    "Customline",
    "E-150",
    "E-250",
    "E-350",
    "E-450",
    "E-Series",
    "EcoSport",
    "Econoline",
    "Edge",
    "Escape",
    "Escort",
    "Excursion",
    "Expedition",
    "Explorer",
    "F-1",
    "F-100",
    "F-150",
    "F-250",
    "F-350",
    "F-450",
    "F-550",
    "F-600",
    "F-650",
    "F-700",
    "F-750",
    "F-800",
    "F-Series",
    "F-Super Duty",
    "Fairlane",
    "Fairmont",
    "Falcon",
    "Festiva",
    "Fiesta",
    "Five Hundred",
    "Flex",
    "Focus",
    "Freestar",
    "Freestyle",
    "Fusion",
    "Galaxie",
    "Gran Torino",
    "Granada",
    "LTD",
    "Lightning",
    "Mainline",
    "Maverick",
    "Model A",
    "Model B",
    "Model T",
    "Mustang",
    "Pickup",
    "Pinto",
    "Police Interceptor",
    "Probe",
    "Ranchero",
    "Ranger",
    "Raptor",
    "Roadster",
    "Sedan",
    "Sport Trac",
    "Super Duty",
    "Taurus",
    "Tempo",
    "Thunderbird",
    "Torino",
    "Transit",
    "Windstar"
] 

gmc_models = [
    "Acadia",
    "Box Truck",
    "Brigadier",
    "Bus",
    "C-Series",
    "C10",
    "C15",
    "C2500",
    "C35",
    "C4",
    "C5500",
    "C6",
    "C7000",
    "C7500",
    "C8500",
    "Caballero",
    "Canyon",
    "Cargo Van",
    "Cheyenne",
    "Custom",
    "Denali",
    "Envoy",
    "Flatbed",
    "G2500",
    "G3500",
    "General",
    "Jimmy",
    "K1000",
    "K15",
    "K20",
    "K2500",
    "K3500",
    "Kodiak",
    "P30",
    "P3500",
    "P60",
    "S15",
    "S1700",
    "Safari",
    "Savana",
    "Sierra",
    "Sonoma",
    "Step Van",
    "Suburban",
    "Syclone",
    "T-Series",
    "T7500",
    "T8500",
    "TC75",
    "Terrain",
    "TopKick",
    "Typhoon",
    "V15",
    "V3500",
    "Vandura",
    "W3500",
    "W4",
    "W55",
    "WS4500",
    "Yukon",
]

jeep_models = [
    "CJ",
    "CJ-2",
    "CJ-2A",
    "CJ-3A",
    "CJ-3B",
    "CJ-5",
    "CJ-6",
    "CJ-7",
    "CJ-8",
    "Cherokee",
    "Comanche",
    "Commander",
    "Compass",
    "DJ-5",
    "FC-170",
    "FCW-170",
    "Gladiator",
    "J-10",
    "J-20",
    "J-300",
    "J-4000",
    "Laredo",
    "Liberty",
    "Limited",
    "M-52",
    "M-715",
    "M-813",
    "MB",
    "MJ",
    "Patriot",
    "Renegade",
    "Rubicon",
    "Sahara",
    "Sport",
    "TJ",
    "Trailhawk",
    "Truck",
    "Wagoneer",
    "Willys",
    "Wrangler",
    "XJ",
    "YJ",
    "ZJ"
]

nissan_models = [
    "240sx",
    "280zx",
    "300zx",
    "350z",
    "370z",
    "altima",
    "armada",
    "cube",
    "frontier",
    "gt-r",
    "hardbody",
    "homy",
    "juke",
    "kicks",
    "leaf",
    "maxima",
    "murano",
    "note",
    "nv",
    "nv1500",
    "nv200",
    "nv2500",
    "nv3500",
    "nx",
    "pao",
    "pathfinder",
    "patrol",
    "pickup",
    "pulsar",
    "quest",
    "rogue",
    "safari",
    "sentra",
    "silvia",
    "skyline",
    "stanza",
    "titan",
    "ud",
    "ud2600",
    "vanette",
    "versa",
    "xterra"
]

ram_models =  [
    "1500",
    "2500", 
    "3500",
    "4500",
    "5500",
    "b150",
    "b350",
    "c-v",
    "cargo-van",
    "chassis-cab",
    "d100",
    "d150",
    "d250",
    "d350",
    "d50",
    "dakota",
    "pickup",
    "power-wagon",
    "promaster",
    "promaster-city",
    "srt-10",
    "trx",
    "van",
    "w150",
    "w250"
]


mazda_models = [
    "626",
    "b-series",
    "cx3",
    "cx-3"
    "cx30",
    "cx5",
    "cx 5",,
    "cx-5,"
    "cx7"
    "cx 7",,
    "cx-7",
    "cx9",
    "cx-9",
    "cx 9",
    "mazda2",
    "mazda3",
    "mazda5",
    "mazda6",
    "mazdaspeed",
    "miata",
    "millenia",
    "mpv",
    "mx6",
    "navajo",
    "pickup",
    "protege",
    "rx4",
    "rx7",
    "rx8",
    "tribute"
]

cadillac_models = [
    "allante",
    "ats",
    "brougham",
    "catera",
    "cimarron",
    "coupe-deville",
    "ct4",
    "ct5",
    "ct6",
    "cts",
    "deville",
    "dts",
    "eldorado",
    "elr",
    "escalade",
    "escalade-esv",
    "escalade-ext",
    "fleetwood",
    "lasalle",
    "sedan-deville",
    "series-62",
    "series-75",
    "seville",
    "sls",
    "srx",
    "sts",
    "xlr",
    "xt4",
    "xt5",
    "xt6",
    "xts"
]

honda_models = [
    "accord",
    "acty",
    "beat",
    "cbr",
    "civic",
    "clarity",
    "cr-v",
    "cr-z",
    "crosstour",
    "crx",
    "del-sol",
    "element",
    "fit",
    "goldwing",
    "hr-v",
    "insight",
    "integra",
    "motorcycle",
    "odyssey",
    "passport",
    "pilot",
    "prelude",
    "ridgeline",
    "s2000",
    "vtx"
]


dodge_models = [
    "a100",
    "aries",
    "aspen",
    "avenger",
    "b-series",
    "caliber",
    "caravan",
    "challenger",
    "charger",
    "coronet",
    "d-series",
    "dakota",
    "dart",
    "daytona",
    "demon",
    "diplomat",
    "durango",
    "dynasty",
    "grand-caravan",
    "intrepid",
    "journey",
    "m-series",
    "magnum",
    "meadowbrook",
    "mirada",
    "monaco",
    "neon",
    "nitro",
    "polara",
    "promaster",
    "raider",
    "ram",
    "ramcharger",
    "rampage",
    "roadrunner",
    "shadow",
    "spirit",
    "sprinter",
    "stealth",
    "stratus",
    "super-bee",
    "viper",
    "w-series",
    "wayfarer"
] 

lexus_models = [
    "ct",
    "es",
    "gs",
    "gx",
    "hs",
    "is",
    "lc",
    "ls",
    "lx",
    "nx",
    "rc",
    "rx",
    "sc",
    "ux"
]


jaguar_models = [
    "e-pace",
    "e-type",
    "f-pace",
    "f-type",
    "i-pace",
    "mk9",
    "s-type",
    "ss100",
    "vanden-plas",
    "x-type",
    "xe",
    "xf",
    "xj",
    "xjs",
    "xk"
]

buick_models = [
    "allure",
    "apollo",
    "cascada",
    "centurion",
    "century",
    "eight",
    "electra",
    "enclave",
    "encore",
    "envision",
    "estate-wagon",
    "grand-national",
    "gs",
    "lacrosse",
    "lesabre",
    "lucerne",
    "model-57",
    "park-avenue",
    "rainier",
    "reatta",
    "regal",
    "rendezvous",
    "riviera",
    "roadmaster",
    "sedanette",
    "series-40",
    "skyhawk",
    "skylark",
    "special",
    "super",
    "terraza",
    "verano",
    "wildcat"
]

chrysler_models = [
    "200",
    "300",
    "300m",
    "60-series",
    "aspen",
    "cirrus",
    "concorde",
    "conquest",
    "cordoba",
    "crossfire",
    "crown-imperial",
    "fifth-avenue",
    "gem",
    "hurst",
    "imperial",
    "lebaron",
    "lhs",
    "model-70",
    "new-yorker",
    "newport",
    "pacifica",
    "prowler",
    "pt-cruiser",
    "royal",
    "sebring",
    "tc-maserati",
    "town-country",
    "valiant",
    "voyager",
    "windsor"
]

SyntaxError: invalid syntax (733040967.py, line 1127)